   Copyright 2016-2024 Wannaphong Phatthiyaphaibun

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.


In [1]:
!wget https://github.com/PyThaiNLP/thai-g2p-wiktionary-corpus/raw/main/wiktionary-23-7-2022-clean.tsv

--2023-01-12 13:38:09--  https://github.com/PyThaiNLP/thai-g2p-wiktionary-corpus/raw/main/wiktionary-23-7-2022-clean.tsv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PyThaiNLP/thai-g2p-wiktionary-corpus/main/wiktionary-23-7-2022-clean.tsv [following]
--2023-01-12 13:38:10--  https://raw.githubusercontent.com/PyThaiNLP/thai-g2p-wiktionary-corpus/main/wiktionary-23-7-2022-clean.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 858538 (838K) [text/plain]
Saving to: ‘wiktionary-23-7-2022-clean.tsv’

wiktionary-23-7-202 100%[===================>] 838.42K  --.-KB/s    in 0.05s   

2023-01-12 

In [2]:
import pandas as pd

In [7]:
df = pd.read_csv("wiktionary-23-7-2022-clean.tsv",sep="\t",names=["grapheme","phoneme"])

In [8]:
df

,grapheme,phoneme
0,ก,k ɔː ˧
1,ก.,k ɔː ˧
2,ก.ค.,k ɔː ˧ . kʰ ɔː ˧
3,ก.ท.ม.,k ɔː ˧ . tʰ ɔː ˧ . m ɔː ˧
4,ก.พ.,k ɔː ˧ . pʰ ɔː ˧
...,...,...
16023,ไฮโซ,h aj ˧ . s oː ˧
16024,ไฮโดรคาร์บอน,h aj ˧ . d r oː ˧ . kʰ aː ˧ . b ɔ n ˥˩
16025,ไฮโดรเจน,h aj ˧ . d r oː ˧ . t͡ɕ eː n ˧
16026,ไฮโดรเจน,h aj ˧ . d r oː ˧ . t͡ɕ e n ˥˩


In [9]:
df.shape

(16028, 2)

In [10]:
df.drop_duplicates().shape

(15956, 2)

In [11]:
df.grapheme.nunique(),df.phoneme.nunique()

(14839, 15250)

In [12]:
df.to_csv('wiktionary_dedup.csv',header=None, index=None, sep='\n')

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
# thank https://github.com/LalitaDeelert/lalita-mt-zhth/blob/main/notebooks/huggingface_tutorial.ipynb
train,test = train_test_split(df, test_size=0.2, random_state=42)
train,val= train_test_split(train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

In [25]:
train.to_csv('train.tsv', index=None, sep='\t')
val.to_csv('val.tsv', index=None, sep='\t')
test.to_csv('test.tsv', index=None, sep='\t')

In [13]:
import sentencepiece as spm

In [15]:
spm.SentencePieceTrainer.train('--input=wiktionary_dedup.csv --character_coverage=1.0 --model_prefix=wiktionary --vocab_size=5000')

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=wiktionary_dedup.csv --character_coverage=1.0 --model_prefix=wiktionary --vocab_size=5000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: wiktionary_dedup.csv
  input_format: 
  model_prefix: wiktionary
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece:

In [16]:
sp = spm.SentencePieceProcessor()

In [17]:
sp.load('wiktionary.model')

True

In [20]:
sp.encode_as_pieces('ผมทำการตัดคำภาษาไทย')

['▁', 'ผม', 'ท', 'ํา', 'การ', 'ตัด', 'คํา', 'ภาษา', 'ไทย']

In [26]:
train_df.phoneme.map(lambda x: len(sp.encode_as_pieces(x))).describe()

NameError: name 'train_df' is not defined